compare between model results and survey data: see how the results change/improve when we use coefficients estimated from our survey

- variables: income, hhsize, number of adults/drivers/workers, household/employment density
- add total number of households in graphs


In [1]:
import os
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path
import plotly.express as px

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# path
## landuse data
p_landuse = "R:/e2projects_two/activitysim/estimation/2017_2019_data/validation_data/survey_outputs/final_land_use.csv"
## survey data
p_model_survey_data = "R:/e2projects_two/activitysim/estimation/2017_2019_data/validation_data/survey_outputs/final_households.csv"
## psrc model results
p_psrc_model_data = "R:/e2projects_two/activitysim/estimation/2017_2019_data/validation_data/model_outputs/pipeline.parquetpipeline/households/mp_households.parquet"


p_acs_auto_ownership = "R:/e2projects_two/activitysim\estimation/2017_2019_data/validation_data/auto_ownership/bg_auto_ownership.csv"
p_maz_bg = "R:/e2projects_two/activitysim\estimation/2017_2019_data/validation_data/auto_ownership/maz_bg_lookup.csv"

## Household data

- income, hh density, employment density grouped into very low, low, medium, medium-high and high

In [2]:
# read data

# land use
land_use = pd.read_csv(p_landuse)

# model data
hh_data_model = pd.read_parquet(p_psrc_model_data).reset_index()
# add weight to model data with all 1
hh_data_model['hh_weight_2017_2019'] = np.repeat(1, len(hh_data_model))
# add employment and household density from landuse data and block group id
hh_data_model = hh_data_model.merge(land_use[['zone_id','log_emptot_1','log_hh_1']],how="left",left_on='home_zone_id',right_on='zone_id').\
    merge(pd.read_csv(p_maz_bg)[['MAZ', 'block_group_id']], how="left",left_on='home_zone_id',right_on='MAZ')


# survey data
hh_data_survey = pd.read_csv(p_model_survey_data).groupby('household_id_elmer').first().reset_index() # remove duplicates
hh_data_survey = hh_data_survey.merge(land_use[['zone_id','log_emptot_1','log_hh_1']],how="left",left_on='home_zone_id',right_on='zone_id').\
    merge(pd.read_csv(p_maz_bg)[['MAZ', 'block_group_id']], how="left",left_on='home_zone_id',right_on='MAZ')

# grouping income, hh density, employment density into very low, low, medium, medium-high and high
var_group = hh_data_model[['income','log_emptot_1','log_hh_1']].quantile([.125, .25, .50, .75])

print(f"household counts \n"
      f"- model results: {len(hh_data_model)}\n"
      f"- survey results: {hh_data_survey['hh_weight_2017_2019'].sum()}\n"
      f"group dividers:\n"
      f"{var_group}")
# var_group['income'].loc[0.125]

household counts 
- model results: 1605263
- survey results: 1505466.8803424172
group dividers:
         income  log_emptot_1  log_hh_1
0.125   24000.0      1.321043  4.434358
0.250   43000.0      2.922615  5.089858
0.500   82000.0      4.726863  5.721498
0.750  135000.0      6.239594  6.457123


In [3]:
# process model and survey data for summary
def data_process(df: pd.DataFrame, var_group_df: pd.DataFrame, data_source: str) -> pd.DataFrame:
    
    def var_grouping(df: pd.DataFrame, group_var:str) -> pd.Series:
        df['temp'] = np.where(df[group_var]<var_group_df[group_var].loc[0.125], 'very low',
                     np.where(df[group_var]<var_group_df[group_var].loc[0.250], 'low',
                     np.where(df[group_var]<var_group_df[group_var].loc[0.500], 'medium',
                     np.where(df[group_var]<var_group_df[group_var].loc[0.750], 'medium-high',
                     np.where(df[group_var]>=var_group_df[group_var].loc[0.750],'high', np.nan)))))
        df['temp'] = df['temp'].astype(pd.CategoricalDtype(['very low', 'low', 'medium', 'medium-high', 'high']))
        return df['temp']
        
        
    # add data source
    df['source'] = data_source
    # add auto_ownership with 4+
    df['auto_ownership_simple'] = np.where(df['auto_ownership']>=4, "4+", df['auto_ownership'])
    # add auto_ownership with 2+
    df['auto_ownership_2'] = np.where(df['auto_ownership']<2, df['auto_ownership'], "2+")
    # add hhsize with 4+
    df['hhsize_simple'] = np.where(df['hhsize']>=4, "4+", df['hhsize'])
    # add num_workers with 4+
    df['num_workers_simple'] = np.where(df['num_workers']>=4, "4+", df['num_workers'])
    # add num_adults with 4+
    df['num_adults_simple'] = np.where(df['num_adults']>=4, "4+",df['num_adults'])
    # add num_drivers with 4+
    df['num_drivers_simple'] = np.where(df['num_drivers']>=4, "4+",df['num_drivers'])

    # add income groups
    df['hhincome_group'] = var_grouping(df, 'income')
    # add hh density groups
    df['hh_density_group'] = var_grouping(df, 'log_hh_1')
    # add employment density groups
    df['emp_density_group'] = var_grouping(df, 'log_emptot_1')

    return df

# match columns and concat all source into hh_data
col_list = ['household_id', 'home_zone_id', 'auto_ownership_simple', 'auto_ownership_2',
            'hhincome_group', 'hhsize_simple', 'hh_density_group', 'emp_density_group',
            'num_workers_simple','num_adults_simple','num_drivers_simple', 'block_group_id',
            'source', 'hh_weight_2017_2019']

# combine both sets of data
hh_data = pd.concat([data_process(hh_data_model,  var_group, "model results")[col_list].copy(),
                     data_process(hh_data_survey, var_group, "survey data")[col_list].copy()])

# hh_data

## Auto ownership across all households

In [4]:
unweight_plot = hh_data.loc[hh_data['source']=="survey data"].\
    groupby(['auto_ownership_simple'])['hh_weight_2017_2019'].count().reset_index()
unweight_plot['source']="unweighted survey"
unweight_plot['hh_weight_2017_2019'] = unweight_plot['hh_weight_2017_2019'].astype('float64')

unweight_plot2 = hh_data.groupby(['source','auto_ownership_simple'])['hh_weight_2017_2019'].\
    sum().reset_index()

df_plot = pd.concat([unweight_plot[['source','auto_ownership_simple','hh_weight_2017_2019']],unweight_plot2]).reset_index(drop=True)
# df_plot.groupby(['source'])['hh_weight_2017_2019'].sum()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hh_weight_2017_2019'].\
        apply(lambda x: 100 * x / float(x.sum()))
df_plot['source'] = df_plot['source'].astype(pd.CategoricalDtype(['model results', 'survey data', 'unweighted survey']))

fig = px.bar(df_plot.sort_values(by=['source']), x="auto_ownership_simple", y="percentage", color="source",
             barmode="group",template="simple_white",
             title="Auto ownership")
fig.update_layout(height=400, width=700, font=dict(size=11))
fig.show()

## Auto ownership distribution within groups

In [5]:
# auto ownership in Income groups
def plot_auto(df:pd.DataFrame, var:str, title_cat:str,sub_name:str):
    print(f"n=\n"
          f"{df.loc[df['source']=='model results',var].value_counts()[df[var].sort_values().unique()]}")
    df_plot = df.groupby(['source',var,'auto_ownership_simple'])['hh_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',var], group_keys=False)['hh_weight_2017_2019'].\
        apply(lambda x: 100 * x / float(x.sum()))

    fig = px.bar(df_plot, x="auto_ownership_simple", y="percentage", color="source",
                 facet_col=var, barmode="group",template="simple_white",
                 title="Auto ownership in each "+ title_cat+ " group")
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text="n of cars")
    fig.update_layout(height=400, width=700, font=dict(size=11))
    fig.show()

### Income

In [6]:
plot_auto(hh_data,'hhincome_group','income level', 'Income')

n=
very low       197507
low            200173
medium         403002
medium-high    402988
high           401593
Name: hhincome_group, dtype: int64


### Household size

In [7]:
plot_auto(hh_data,'hhsize_simple','household size', 'HH size')

n=
1     452162
2     533682
3     254336
4+    365083
Name: hhsize_simple, dtype: int64


### Number of adults

In [8]:
plot_auto(hh_data,'num_adults_simple','number of adults','num adults')

n=
0         75
1     504125
2     762899
3     240181
4+     97983
Name: num_adults_simple, dtype: int64


### Number of workers

In [9]:
plot_auto(hh_data,'num_workers_simple','number of workers','num workers')

n=
0     355309
1     637690
2     504466
3      99225
4+      8573
Name: num_workers_simple, dtype: int64


### Number of drivers

In [10]:
plot_auto(hh_data.loc[hh_data['num_drivers_simple']!="0"],'num_drivers_simple','number of drivers','num drivers')

n=
1     493312
2     708568
3     274525
4+    128847
Name: num_drivers_simple, dtype: int64


### Household density

In [11]:
plot_auto(hh_data.dropna(subset=['hh_density_group']),'hh_density_group','household density','density')

n=
very low       200617
low            200583
medium         401416
medium-high    401303
high           401344
Name: hh_density_group, dtype: int64


### Employment density

In [12]:
plot_auto(hh_data.dropna(subset=['emp_density_group']),'emp_density_group','employment density','density')

n=
very low       200546
low            200554
medium         401485
medium-high    401336
high           401342
Name: emp_density_group, dtype: int64


## Validate auto ownership with ACS vehicle ownership data

- TO-DO: get auto ownership values [0,1,2,3,4+] from ACS data
- household counts in ACS data

In [13]:
print(f"n=\n"
      f"{hh_data.loc[hh_data['source']=='model results','auto_ownership_2'].value_counts()[hh_data['auto_ownership_2'].sort_values().unique()]}")
      
df = hh_data.groupby(['source','block_group_id','auto_ownership_2'])['hh_weight_2017_2019'].sum().reset_index()
# df_plot = df_plot.loc[df_plot['auto_ownership_2']!="-1"]

df['percentage'] = df.groupby(['source','block_group_id'], group_keys=False)['hh_weight_2017_2019'].\
    apply(lambda x: 100 * x / float(x.sum()))

# acs auto ownership data
acs_auto_ownership = pd.read_csv(p_acs_auto_ownership)[['cars_none_control', 'cars_one_control', 'cars_two_or_more_control', 'block_group_id']]

# calculate percentage of households having 0, 1 or 2+ vehicle(s) in each block group
acs_auto_ownership['total'] = acs_auto_ownership['cars_one_control'] + acs_auto_ownership['cars_two_or_more_control'] + acs_auto_ownership['cars_none_control']
acs_auto_ownership['0'] = 100 * acs_auto_ownership['cars_none_control']/acs_auto_ownership['total']
acs_auto_ownership['1'] = 100 * acs_auto_ownership['cars_one_control']/acs_auto_ownership['total']
acs_auto_ownership['2+'] = 100 * acs_auto_ownership['cars_two_or_more_control']/acs_auto_ownership['total']
acs_auto_ownership['source'] = "acs data"
bg_auto_ownership = acs_auto_ownership[['source','block_group_id','0','1','2+']]
bg_auto_ownership = pd.melt(bg_auto_ownership, id_vars=['source','block_group_id'], value_vars=['0','1','2+'], var_name='auto_ownership_2',value_name='percentage')

# combine both sets of data
col_list = ['source','block_group_id','auto_ownership_2','percentage']
bg_auto_ownership = pd.concat([df[col_list].copy(),
                               bg_auto_ownership[col_list].copy()])

# bg_auto_ownership

n=
0     208607
1     439193
2+    957463
Name: auto_ownership_2, dtype: int64


- scatterplot

In [14]:
df_plot = pd.pivot(bg_auto_ownership, index=['block_group_id','auto_ownership_2'], columns='source', values='percentage').reset_index()



fig = px.scatter(df_plot, x="acs data", y="model results", trendline="ols", trendline_color_override='rgb(136, 136, 136)',
                 template="plotly_white",
                 facet_col='auto_ownership_2', height=400, width=1000,
                 title="Auto ownership model results validation with acs data")
fig.update_xaxes(dtick=20)
fig.update_yaxes(dtick=20,range=[0, 100])
fig.update_traces(marker_size=3)
fig.update_layout(height=400, width=700, font=dict(size=11))
fig.show()
